# CSCE 5290 - Final Project
## Pre-trained model evaluation (BART, T5)
Dan Waters (danwaters@my.unt.edu)

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 13.4 MB/s 
     |████████████████████████████████| 895 kB 51.5 MB/s 
     |████████████████████████████████| 3.3 MB 51.8 MB/s 
     |████████████████████████████████| 59 kB 7.7 MB/s 
     |████████████████████████████████| 596 kB 69.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [3]:
# Get the data (not the one with the start tokens)
!gdown --id 17u3TvSpRq17mFVJ1pEKf6D9fYgBFj4lI

Downloading...
From: https://drive.google.com/uc?id=17u3TvSpRq17mFVJ1pEKf6D9fYgBFj4lI
To: /content/cnn_cleaned_test_10k.csv
100% 6.52M/6.52M [00:00<00:00, 24.7MB/s]


In [4]:
import pandas as pd
cnn_df = pd.read_csv('cnn_cleaned_test_10k.csv')
cnn_df = cnn_df[['text', 'summary']]
cnn_df.head(5)

,text,summary
0,ever noticed how plane seats appear to be gett...,experts question if packed out planes are putt...
1,a drunk teenage boy had to be rescued by secur...,drunk teenage boy climbed into lion enclosure ...
2,dougie freedman is on the verge of agreeing ne...,nottingham forest are close to extending dougi...
3,liverpool target neto is also wanted by psg an...,fiorentina goalkeeper neto has been linked wit...
4,this is the moment that crew of firefighters s...,giant pig fell into the swimming pool at his h...


In [5]:
article = cnn_df.iloc[0]['text']
article

'ever noticed how plane seats appear to be getting smaller and smaller with increasing numbers of people taking to the skies some experts are questioning if having such packed out planes is putting passengers at risk they say that the shrinking space on aeroplanes is not only uncomfortable it putting our health and safety in danger more than squabbling over the arm rest shrinking space on planes putting our health and safety in danger this week u.s consumer advisory group set up by the department of transportation said at public hearing that while the government is happy to set standards for animals flying on planes it doesn stipulate minimum amount of space for humans in world where animals have more rights to space and food than humans said charlie leocha consumer representative on the committee.\xa0 it is time that the dot and faa take stand for humane treatment of passengers but could crowding on planes lead to more serious issues than fighting for space in the overhead lockers cra

### Write the results to files to compare with ROUGE. 

Limit of 100 documents to conserve time.

In [6]:
# Summarize a bunch of articles
score_max_count = 100

with open('cnn_pred_bart_test.txt', 'w') as f_pred:
  for i, c in cnn_df.iterrows():
    text = cnn_df.iloc[i]['text']
    s = summarizer(text, max_length=250, min_length=25, do_sample=False)
    f_pred.write(s[0]['summary_text'])
    if i >= score_max_count:
      break

Your max_length is set to 250, but you input_length is only 134. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 179. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 180. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 162. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 239. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 213. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 250, but you input_length is only 194. You might c

In [7]:
with open('cnn_ref_test.txt', 'w') as f_ref:
  for i, c in cnn_df.iterrows():
    summary = cnn_df.iloc[i]['summary']
    f_ref.write(summary)
    if i >= score_max_count:
      break

In [8]:
!pip install rouge

In [9]:
from rouge import FilesRouge
import sys
fr = FilesRouge()
#sys.setrecursionlimit(50 * 50 + 10)
scores = fr.get_scores('cnn_pred_bart_test.txt', 'cnn_ref_test.txt', avg=True)

In [10]:
scores

{'rouge-1': {'f': 0.5633467980847118,
  'p': 0.483278955954323,
  'r': 0.6752136752136753},
 'rouge-2': {'f': 0.2183155186319587,
  'p': 0.17847078715010334,
  'r': 0.28106508875739644},
 'rouge-l': {'f': 0.5005942428196772, 'p': 0.4294453507340946, 'r': 0.6}}

### Compare with Google's T5 transformer (pretrained on some data including CNN/Dailymail)
see: https://huggingface.co/transformers/task_summary.html

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)

outputs = model.generate(inputs["input_ids"], 
                         max_length=250, min_length=25, length_penalty=2.0, 
                         num_beams=4, early_stopping=True)

print(tokenizer.decode(outputs[0]))

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

<pad> u.s consumer advisory group set up by the department of transportation said at public hearing that the government is happy to set standards for animals flying on planes but doesn't stipulate minimum amount of space for humans. many economy seats on united airlines have 30 inches of room while some airlines offer as little as 28 inches.</s>


In [12]:
# Summarize a bunch of articles
from tqdm import tqdm
score_max_count = 100

with open('cnn_pred_t5_test.txt', 'w') as f_pred:
  for i, c in tqdm(cnn_df.iterrows(), total = cnn_df.shape[0]):
    text = cnn_df.iloc[i]['text']

    inputs = tokenizer("summarize: " + text, return_tensors="pt", 
                       max_length=512, truncation=True)
    
    outputs = model.generate(inputs["input_ids"], 
                         max_length=250, min_length=25, length_penalty=2.0, 
                         num_beams=4, early_stopping=True)
    
    s = tokenizer.decode(outputs[0]).replace('<pad>', '').replace('</s>', '')

    f_pred.write(s)
    if i >= score_max_count:
      break

  3%|▎         | 100/2961 [10:27<4:59:25,  6.28s/it]


In [14]:
scores = fr.get_scores('cnn_pred_t5_test.txt', 'cnn_ref_test.txt', avg=True)
scores

{'rouge-1': {'f': 0.5740428557025695,
  'p': 0.4970809007506255,
  'r': 0.6792022792022792},
 'rouge-2': {'f': 0.21848541383681777,
  'p': 0.18130366900858705,
  'r': 0.2748520710059172},
 'rouge-l': {'f': 0.5172164651415295,
  'p': 0.44787322768974147,
  'r': 0.611965811965812}}